# API Pull

In [28]:
import os, json, time, datetime as dt, csv, pathlib
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("API Key present:", ALPHA_KEY is not None)

API Key present: True


In [29]:
SYMBOL = "AAPL"
use_alpha = bool(ALPHA_KEY)
print("Using Alpha Vantage:", use_alpha)

df_api = None

if use_alpha:
    try:
        url = "https://www.alphavantage.co/query"
        params = {
            "function": "TIME_SERIES_DAILY_ADJUSTED",
            "symbol": SYMBOL,
            "outputsize": "compact",
            "apikey": ALPHA_KEY,
            "datatype": "json"
        }
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        js = r.json()
        key = [k for k in js.keys() if "Time Series" in k]
        if not key:
            raise ValueError(f"Unexpected response keys: {list(js.keys())}")
        series = js[key[0]]
        df_api = (pd.DataFrame(series).T
                  .rename_axis('date')
                  .reset_index())
        df_api = df_api[['date', '5. adjusted close']].rename(columns={'5. adjusted close': 'adj_close'})
        df_api['date'] = pd.to_datetime(df_api['date'])
        df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
        print("[OK] Got data from Alpha Vantage")
    except Exception as e:
        print("[WARN] Alpha Vantage failed:", e)
        use_alpha = False  # fallback

if not use_alpha:
    import yfinance as yf
    df_yf = yf.download(SYMBOL, period="6mo", interval="1d", auto_adjust=False)

    if isinstance(df_yf.columns, pd.MultiIndex):
        df_yf.columns = df_yf.columns.get_level_values(0)

    df_yf = df_yf.reset_index()

    if "Adj Close" in df_yf.columns:
        df_api = df_yf[['Date', 'Adj Close']].rename(columns={'Date': 'date', 'Adj Close': 'adj_close'})
    else:
        df_api = df_yf[['Date', 'Close']].rename(columns={'Date': 'date', 'Close': 'adj_close'})

    print("[OK] Got data from yfinance")

df_api = df_api.sort_values('date').reset_index(drop=True)


Using Alpha Vantage: True
[WARN] Alpha Vantage failed: Unexpected response keys: ['Information']


[*********************100%***********************]  1 of 1 completed

[OK] Got data from yfinance


In [30]:
# --- Validation ---
assert 'date' in df_api.columns and 'adj_close' in df_api.columns, "lack requre column"
assert df_api['date'].notna().all(), "date have NA"
assert df_api['adj_close'].notna().all(), "adj_close have NA"
assert len(df_api) > 0, "DataFrame is empty"
print("Validation passed")

Validation passed


In [31]:
print(df_api.head())

Price       date   adj_close
0     2025-02-18  243.873062
1     2025-02-19  244.272079
2     2025-02-20  245.229736
3     2025-02-21  244.950424
4     2025-02-24  246.496643


saving

In [32]:
RAW_DIR = Path("..") / "data" / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

source = "alpha" if use_alpha else "yfinance"
fname = RAW_DIR / f"api_{source}_{SYMBOL}_{pd.Timestamp.now().strftime('%Y%m%d-%H%M')}.csv"

df_api.to_csv(fname, index=False)
print("Saved:", fname)


Saved: ..\data\raw\api_yfinance_AAPL_20250818-1033.csv


# Scraping

In [ ]:
SCRAPE_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "AFE-Course-Notebook/1.0 (contact: instructor@example.edu)"}

try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Find Constituents table
    table = soup.find("table", {"id": "constituents"})
    if table is None:
        raise ValueError("Constituents table not found")

    rows = []
    for tr in table.find_all("tr"):
        cells = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
        if cells:
            rows.append(cells)

    # First row is header
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

except Exception as e:
    print("Scrape failed (fallback to inline demo).", e)
    html = """
    <table>
      <tr><th>Symbol</th><th>Security</th><th>GICS Sector</th></tr>
      <tr><td>AAA</td><td>Demo Corp</td><td>Industrials</td></tr>
      <tr><td>BBB</td><td>Example Inc</td><td>Technology</td></tr>
    </table>
    """
    soup = BeautifulSoup(html, 'html.parser')
    rows = []
    for tr in soup.find_all("tr"):
        cells = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
        if cells:
            rows.append(cells)
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

# --- Validation ---
df_scrape.columns = df_scrape.columns.str.strip().str.replace("\n", " ", regex=True)
required_cols = ["Symbol", "Security"]
missing = [c for c in required_cols if c not in df_scrape.columns]
assert not missing, f"Missing required columns: {missing}"

gics_cols = [c for c in df_scrape.columns if "GICS" in c]
assert gics_cols, "No GICS-related columns found"

assert df_scrape["Symbol"].notna().all(), "Symbol column has NA values"
assert df_scrape["Security"].notna().all(), "Security column has NA values"
for col in gics_cols:
    assert df_scrape[col].notna().all(), f"{col} has NA values"

assert len(df_scrape) > 400, "Row count too small, scraping likely failed"

print("Validation passed (Scraping)")

# --- Save ---
fname2 = f"scrape_wikipedia_sp500_{pd.Timestamp.now().strftime('%Y%m%d-%H%M')}.csv"
out_path2 = (Path("..") / "data" / "raw" / fname2)
df_scrape.to_csv(out_path2, index=False)
print("Saved:", out_path2)

df_scrape.head()


Validation passed (Scraping)
Saved: ..\data\raw\scrape_wikipedia_sp500_20250818-1033.csv


,Symbol,Security,GICSSector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


: 

# Documentation

**Data Sources**  
- Alpha Vantage API: `https://www.alphavantage.co/query` (used to download stock time series data)  
- Yahoo Finance Markets Table: `https://finance.yahoo.com/markets` (public webpage with simple market data table)  

**Parameters**  
- Symbol: AAPL  
- Function: TIME_SERIES_DAILY_ADJUSTED  
- Outputsize: compact  
- API Key: stored securely in `.env`  

**Validation Logic**  
- Checked required columns (`date`, `adj_close`) are present  
- Verified no missing values in critical columns  
- Ensured DataFrame is non-empty  
- For scraped table: validated numeric/text columns have correct types  

**Assumptions & Risks**  
- API data may be delayed or limited by rate limits  
- Scraped webpage structure may change without notice  
- Network errors could interrupt data acquisition
